In [1]:
import pandas as pd
import numpy as np

url = 'https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/resstock_dataset_2024.1/resstock_tmy3/metadata_and_annual_results/national/parquet/Baseline/baseline_metadata_and_annual_results.parquet'

df = pd.read_parquet(url)

# df

In [2]:
# df.describe()


In [3]:
# from utils import convert_categorical
def convert_categorical(X: pd.DataFrame):
    category_cols = X.select_dtypes(exclude=np.number).columns.tolist()

    # Convert to Pandas category columns
    for col in category_cols:
        X[col] = X[col].astype('category')
    return X


# df = convert_categorical(df)

in_cols = [col for col in df.columns if col.startswith('in')]
out_cols = [col for col in df.columns if col.startswith('out')]
df['out.electricity.total.energy_consumption.kwh'] = df['out.electricity.total.energy_consumption.kwh'].clip(0,None)
X, y = df[in_cols], df['out.electricity.total.energy_consumption.kwh']

X = convert_categorical(X)
X.describe()

/tmp/ipykernel_16450/1232770359.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  X.loc[col] = X[col].astype('category')
/tmp/ipykernel_16450/1232770359.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[col] = X[col].astype('category')
/tmp/ipykernel_16450/1232770359.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the con

KeyboardInterrupt: 

In [ ]:
## Drop Irrelevant columns and split columns that contain multiple features 

def drop_ignored_columns(X: pd.DataFrame):
    column_plan_df = pd.read_csv('column_plan.csv')
    to_drop = []
    for index, row in column_plan_df.iterrows():
        field = row['field_name']
        keep = row['keep_for_model']
        if keep == "No" and field in X:
            to_drop.append(field)
    X = X.drop(columns=to_drop)
    return X

def str_to_int(text:str):
    if text == "None":
        return 0
    else:
        return int(text)

def split_columns(X:pd.DataFrame):
    # Split possible outcomes by comma, leakage and insulation are two features
    X[['in.duct_leakage','in.duct_insulation']] = X['in.duct_leakage_and_insulation'].str.split(', ',expand=True)
    X = X.drop(columns=['in.duct_leakage_and_insulation'])

    # Columns that are integers with "None" as 0
    X['in.geometry_building_number_units_mf'] = X['in.geometry_building_number_units_mf'].apply(str_to_int)
    X['in.geometry_building_number_units_sfa'] = X['in.geometry_building_number_units_sfa'].apply(str_to_int)

    # Convert string int columns to numeric
    X['in.geometry_building_number_units_mf'] = pd.to_numeric(X['in.geometry_building_number_units_mf'])
    X['in.geometry_building_number_units_sfa'] = pd.to_numeric(X['in.geometry_building_number_units_sfa'])
    X['in.geometry_stories'] = pd.to_numeric(X['in.geometry_stories'])
    X['in.bedrooms'] = pd.to_numeric(X['in.bedrooms'])

    # Split "insulation_wall into two columns - insulation rating, and wall material"
    X['in.wall_type'] = X['in.insulation_wall'].str.split(',').str[0]
    X['in.wall_insulation_rating'] = X['in.insulation_wall'].str.split(', ').str[-1]
    X = X.drop(columns=['in.insulation_wall'])
    
    # Retain only heating type
    X['in.hvac_heating_efficiency'] = X['in.hvac_heating_efficiency'].str.split(',').str[0]

    # Convert compass direction to two-dimensional vector for each feature
    X['in.orientation_northness'] = X['in.orientation']
    X['in.orientation_eastness'] = X['in.orientation']
    X['in.pv_orientation_northness'] = X['in.pv_orientation']
    X['in.pv_orientation_eastness'] = X['in.pv_orientation']
    X = X.drop(columns=['in.orientation', 'in.pv_orientation'])
    return X




def preprocess_columns(X: pd.DataFrame):
    # X = convert_categorical(X)
    X = drop_ignored_columns(X)
    X = split_columns(X)
    X = convert_categorical(X)
    return X

In [ ]:
# Add encodings to columns that were just split

custom_binary_cols = ['in.wall_type', 'in.hvac_heating_efficiency']

# Manual Oridinal mappings from columns that were split above
# All other ordinal mapping schemes are in "column_plan.csv"
custom_ordinal_mappings = [
    {
        'col':'in.duct_leakage',
        'mapping':{r'20% Leakage': 20, r'30% Leakage': 30, r'0% Leakage': 0, r'10% Leakage': 10, 'None': -10}
    },
    {
        'col':'in.duct_insulation',
        'mapping':{'Uninsulated': 0, 'R-4': 4, 'R-6': 6, 'R-8': 8, None: -10}},
    {
        'col':'in.wall_insulation_rating',
        'mapping':{'Uninsulated': 0, 'R-11': 11, 'R-15': 15, 'R-19': 19, 'R-7': 7,}
    },
    {
        'col':'in.orientation_northness',
        'mapping':{'North': 4, 'Southwest': 1, 'West': 2, 'Northeast': 3, 'South': 0, 'Northwest': 3, 'East': 2, 'Southeast': 1, "None": 2}
    },
    {
        'col':'in.orientation_eastness',
        'mapping':{'North': 2, 'Southwest': 1, 'West': 0, 'Northeast': 3, 'South': 2, 'Northwest': 1, 'East': 4, 'Southeast': 3, "None": 2}
    },
    {
        'col':'in.pv_orientation_northness',
        'mapping':{'North': 4, 'Southwest': 1, 'West': 2, 'Northeast': 3, 'South': 0, 'Northwest': 3, 'East': 2, 'Southeast': 1, "None": 2}
    },
    {
        'col':'in.pv_orientation_eastness',
        'mapping':{'North': 2, 'Southwest': 1, 'West': 0, 'Northeast': 3, 'South': 2, 'Northwest': 1, 'East': 4, 'Southeast': 3, "None": 2}
    }
]

# X.astype({'in.wall_type':'category', 'in.hvac_heating_efficiency':'category', 'in.duct_leakage':'category', 'in.duct_insulation':'category', 'in.wall_insulation_rating':'category', 'in.orientation_northness':'category', 'in.orientation_eastness':'category', 'in.pv_orientation_northness':'category', 'in.pv_orientation_eastness':'category'})

X = preprocess_columns(X)
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 


In [ ]:
import json

observed_enums = []
num_enums = []
cols = []
types = []
means = {}
types_dict = {}
for col in X.columns:
    observed_enum = X[col].unique()
    joined = '|'.join(str(v) for v in observed_enum)
    observed_enums.append(joined)
    num_enums.append(len(observed_enum))
    cols.append(col)
    types.append(X[col].dtype)
    if X[col].dtype == 'category':
        means[col] = X[col].mode()[0]
    else:
        means[col] = X[col].mean()
    types_dict[col] = str(X[col].dtype)

X_columns = pd.DataFrame({'column': cols, 'type': types, 'enumerations': observed_enums, 'num_enumerations': num_enums})
X_columns.to_csv('X_columns.csv')

avg_home = {'means': means, 'types': types_dict}

with open("avg_home.json", "w") as outfile: 
    json.dump(avg_home, outfile)

In [ ]:
# import json

# means = {}
# types = {}
# for column in X:
#     if df[column].dtype == 'category':
#         means[column] = df[column].mode()[0]
#     else:
#         means[column] = df[column].mean()
#     types[column] = str(df[column].dtype)

# avg_home = {'means': means, 'types': types}

# with open("avg_home.json", "w") as outfile: 
#     json.dump(avg_home, outfile)


In [ ]:
'''
Automatically build ordinal mapping format from a string delimited with "|" (csv safe character)
'''
def get_ordinal_mapping(text:str):
    mapping = {}
    split_array = text.split("|")
    for i, item in enumerate(split_array):
        if i % 2 == 1:
            mapping[split_array[i-1]] = int(split_array[i])
    return mapping

In [ ]:
## Build Category Encoders from those specified column_plan.csv

import category_encoders as ce

column_plan_df = pd.read_csv('column_plan.csv')

# Binary
binary_fields = column_plan_df[column_plan_df['encoder'] == 'Binary']['field_name'].to_list()
binary_fields += custom_binary_cols
binary_encoder = ce.BinaryEncoder(cols=binary_fields)

# Ordinal
full_mapping = []
for index, row in column_plan_df.iterrows():
    if row['encoder'] != 'Ordinal':
        continue
    mapping_dict = {}
    mapping_dict['col'] = row['field_name']
    mapping_dict['mapping'] = get_ordinal_mapping(row['label_encoder_dict'])
    full_mapping.append(mapping_dict)

full_mapping += custom_ordinal_mappings
ordinal_fields = []
for mapping in full_mapping:
    ordinal_fields.append(mapping['col'])
ordinal_encoder = ce.OrdinalEncoder(cols=ordinal_fields, mapping=full_mapping)

# Catboost
catboost_fields = column_plan_df[column_plan_df['encoder'] == 'CatBoost']['field_name'].to_list()
catboost_encoder = ce.CatBoostEncoder(cols=catboost_fields)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pickle

# Build pipeline

preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_encoder, binary_fields),
        ('ordinal', ordinal_encoder, ordinal_fields),
        ('catboost', catboost_encoder, catboost_fields),
    ],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler())
])

# pipeline.fit(X, y)

# with open('pipeline.pkl', 'wb') as f:
#     pickle.dump(pipeline, f)

In [ ]:
from sklearn.model_selection import train_test_split
X = preprocess_columns(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:


X_train_clean = pipeline.fit(X_train, y_train)

with open('pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [ ]:


with open('pipeline.pkl', 'rb') as f:
    loaded_pipeline = pickle.load(f)

X_train_clean = loaded_pipeline.transform(X_train)

In [ ]:
from xgboost import XGBRegressor


model = XGBRegressor(
    objective = 'reg:squarederror',
    tree_method = 'hist',
    max_depth = 2,
    n_estimators = 50,
    learning_rate = .1,
)

model.fit(X_train_clean, y_train)


In [ ]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import GridSearchCV

# estimator = XGBRegressor(
#     objective = 'reg:squarederror',
#     tree_method = 'hist',
#     enable_categorical = True
#     # device="cuda"
# )

# parameters = {
#     'max_depth': range (2),
#     'n_estimators': np.linspace(50, 250, 1, dtype=int),
#     'learning_rate': np.logspace(-2, -.3, 1)
# }

# grid_search = GridSearchCV(
#     estimator=estimator,
#     param_grid=parameters,
#     scoring = 'neg_root_mean_squared_error',
#     n_jobs = -1,
#     cv = 2,
#     verbose=True
# )

# grid_search.fit(X_train, y_train)
# results_df = pd.DataFrame(grid_search.cv_results_)
# model = grid_search.best_estimator_
# results_df

In [ ]:


# for index, row in column_plan_df.iterrows():
#     field = row['field_name']
#     if field in df.columns:
#         observed_enum = df[field].unique()
#         actual_enum =  set(observed_enum) #| set(allowable_enum)
#         if len(actual_enum) > 1:
#             joined = '|'.join(str(v) for v in actual_enum)
#             actual_enums.append(joined)
#             num_enums.append(len(actual_enum))
#         else:
#             actual_enums.append(list(actual_enum)[0])
#             num_enums.append(len(actual_enum))
#     else:
#         actual_enums.append("Not observed")
#         num_enums.append("Not observed")

In [ ]:
# One-Time function used to get actual enumerations of the baseline data

# column_plan_df = pd.read_csv('column_planning.csv')
# column_plan_df.drop(column_plan_df.columns[column_plan_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
# # def create_actual_enumerations(cp_df, x_df):
# actual_enums = []
# num_enums = []
# for index, row in column_plan_df.iterrows():
#     field = row['field_name']
#     if field in df.columns:
#         observed_enum = df[field].unique()
#         actual_enum =  set(observed_enum) #| set(allowable_enum)
#         if len(actual_enum) > 1:
#             joined = '|'.join(str(v) for v in actual_enum)
#             actual_enums.append(joined)
#             num_enums.append(len(actual_enum))
#         else:
#             actual_enums.append(list(actual_enum)[0])
#             num_enums.append(len(actual_enum))
#     else:
#         actual_enums.append("Not observed")
#         num_enums.append("Not observed")



# column_plan_df['actual_enumerations'] = pd.Series(actual_enums)
# column_plan_df['num_enumerations'] = pd.Series(num_enums)
# # column_plan_df.apply(lambda x: create_actual_enumerations(), axis=1)
# # for index, row in column_plan_df.iterrows():
# #     # allowable_enum = row['allowable_enumerations_baseline']
# #     # allowable_enum = allowable_enum.split("|")
# #     field = row['field_name']
# #     if field in df:
# #         observed_enum = df[field].unique()
# #         actual_enum =  set(observed_enum) #| set(allowable_enum)
# #         if len(actual_enum) > 1:
# #             joined = '|'.join(str(v) for v in actual_enum)
# #             column_plan_df.loc[index, 'actual_enumerations'] = joined
# #         else:
# #             column_plan_df.loc[index, 'actual_enumerations'] = actual_enum
# #     else:
# #         column_plan_df.loc[index, 'actual_enumerations'] = "Not observed"

# column_plan_df.to_csv('column_plan.csv', index=False)
# column_plan_df
# df['in.clothes_dryer'].unique()

In [ ]:
from sklearn.metrics import mean_squared_log_error

X_test_clean = pipeline.transform(X_test)
preds = model.predict(X_test_clean)
preds = preds.clip(0,None)
rmse = mean_squared_log_error(y_test, preds)

print(f"RMSE of the base model: {rmse:.5f}")

In [ ]:
with open('avg_home.json') as f: 
    data = f.read() 
    
combined_dicts = json.loads(data)
# avg_home_types = avg_home['types']
# avg_home_means = avg_home['means']
avg_home_df = pd.DataFrame(combined_dicts['means'], index=[0])
for column in avg_home_df:
    avg_home_df[column] = avg_home_df[column].astype(combined_dicts['types'][column])
avg_home_df.dtypes
# avg_home = pd.read_json('means.json', orient='records', typ='series')
# avg_home = avg_home.to_frame().T
# avg_home

In [ ]:
var = 'in.ashrae_iecc_climate_zone_2004'
val = '2A'

old_type = str(avg_home_df[var].dtype)
if old_type == 'category' and val not in avg_home_df[var].cat.categories:
    avg_home_df[var] = avg_home_df[var].cat.add_categories([val])

# avg_home_df['in.ashrae_iecc_climate_zone_2004'].cat.add_categories('2A')
avg_home_df.at[0, var] = val
avg_home_df[var] = avg_home_df[var].astype(old_type)
avg_home_df.dtypes


In [ ]:
pred = model.predict(avg_home_df)
pred

In [ ]:
from xgboost import plot_importance
plot_importance(model, max_num_features=10) # top 10 most important features

In [ ]:
booster = model.get_booster()
booster.save_model('model_baseline_alabama.json')

In [ ]:
data_dict_df = pd.read_csv('data_dictionary.csv', sep='\t').iloc[:, :6].set_index('field_name')
data_dict_df['allowable_enumerations_baseline'] = data_dict_df['allowable_enumerations_baseline'].str.split('|')
# data_dict_df = data_dict_df.replace({"NaN": np.nan})

# Convert the DataFrame to JSON
json_data = data_dict_df.to_json(orient='index')

# Optionally, save the JSON data to a file
with open('home-energy/data_dictionary.json', 'w') as json_file:
    json_file.write(json_data)

data_dict_df

In [ ]:
data_dict_df.describe()